# Setup
---

In [1]:
import pandas as pd

from regex import regex
from preprocess import read_bills, preprocess_all
from util import split_ttf

In [2]:
bills = read_bills('../lab1/data')

In [3]:
df = preprocess_all(bills)

Could not resolve header for 1996_400.txt


In [4]:
df['is_amendment'].describe()

count     1178
unique       2
top       True
freq       610
Name: is_amendment, dtype: object

In [6]:
train, test, validation = split_ttf(df)

In [8]:
print('train      {}'.format(len(train)))
print('test       {}'.format(len(test)))
print('validation {}'.format(len(validation)))

train      706
test       235
validation 237
